In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pymongo
import time
import random
import pdb


### Make a list of the urls of the categories in the forum

In [3]:
user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
html_page = requests.get('https://curltalk.naturallycurly.com/', headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
soup = BeautifulSoup(html_page.content, 'html.parser')
link_list_forum_top = []
for link in soup.find_all('a'):
    link_list_forum_top.append(link.get('href'))
categories = [s for s in link_list_forum_top if (("categories" in s) and ('https' in s))]
categories[0:]

['https://curltalk.naturallycurly.com/categories/announcements',
 'https://curltalk.naturallycurly.com/categories/newbies-info',
 'https://curltalk.naturallycurly.com/categories/introductions',
 'https://curltalk.naturallycurly.com/categories/curlmart-news',
 'https://curltalk.naturallycurly.com/categories/feedback',
 'https://curltalk.naturallycurly.com/categories/c2',
 'https://curltalk.naturallycurly.com/categories/c3',
 'https://curltalk.naturallycurly.com/categories/c4',
 'https://curltalk.naturallycurly.com/categories/general-discussion-about-curly-hair',
 'https://curltalk.naturallycurly.com/categories/salon-stylist-chatter',
 'https://curltalk.naturallycurly.com/categories/transitioning',
 'https://curltalk.naturallycurly.com/categories/thin-thinning-hair',
 'https://curltalk.naturallycurly.com/categories/readers-product-reviews',
 'https://curltalk.naturallycurly.com/categories/hall-of-shame-discussion',
 'https://curltalk.naturallycurly.com/categories/hall-of-fame-discussion'

#### Get the discussion topic urls for the first x pages of the category

In [53]:
# %%time

link_listdiscussion = []
for i in range(0, 100):
    url = f'{categories[26]}/p{i}'
    print(url)
    user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
    html_page = requests.get(url, headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
     
    # Check status code for an appropriate response fromt the API
    status = html_page.status_code
    if status != 200:
        print(f'Error improper response code. Code is {status}')
        #break
    soup = BeautifulSoup(html_page.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
    page1_of_discussion = []
    #time.sleep(random.randint(20,45))
    for link in soup.find_all('a'):
        link_listdiscussion.append(link.get('href'))
        topics = [s for s in link_listdiscussion if (("/discussion" in s) and ('https' in s))]

print(len(topics))     
# Loop through all the topics found for each catagory        
list_for_mongo = []
count = 1
for topic in topics:
    url2 = topic
    user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
    html_page2 = requests.get(url2, headers = {'User-Agent': user_agent} )
    
    if status != 200:
        print(f'Error improper response code. Code is {status}')
        #break
        
    soup2 = BeautifulSoup(html_page2.content, 'html.parser')
    signatures = soup2.find_all('div', class_="Signature UserSignature userContent")
    
    for i in range(0,len(signatures)):
        sig = {}
        sig['signature'] = soup2.find_all('div', class_="Signature UserSignature userContent")[i].get_text()
        list_for_mongo.append(sig)
#     time.sleep(random.randint(1,5))
    print(count)
    count += 1
    

https://curltalk.naturallycurly.com/categories/curly-teens/p0
https://curltalk.naturallycurly.com/categories/curly-teens/p1
https://curltalk.naturallycurly.com/categories/curly-teens/p2
https://curltalk.naturallycurly.com/categories/curly-teens/p3
https://curltalk.naturallycurly.com/categories/curly-teens/p4
https://curltalk.naturallycurly.com/categories/curly-teens/p5
https://curltalk.naturallycurly.com/categories/curly-teens/p6
https://curltalk.naturallycurly.com/categories/curly-teens/p7
https://curltalk.naturallycurly.com/categories/curly-teens/p8
https://curltalk.naturallycurly.com/categories/curly-teens/p9
https://curltalk.naturallycurly.com/categories/curly-teens/p10
https://curltalk.naturallycurly.com/categories/curly-teens/p11
https://curltalk.naturallycurly.com/categories/curly-teens/p12
https://curltalk.naturallycurly.com/categories/curly-teens/p13
https://curltalk.naturallycurly.com/categories/curly-teens/p14
https://curltalk.naturallycurly.com/categories/curly-teens/p15
ht

Error improper response code. Code is 404
20
Error improper response code. Code is 404
21
Error improper response code. Code is 404
22
Error improper response code. Code is 404
23
Error improper response code. Code is 404
24
Error improper response code. Code is 404
25
Error improper response code. Code is 404
26
Error improper response code. Code is 404
27
Error improper response code. Code is 404
28
Error improper response code. Code is 404
29
Error improper response code. Code is 404
30
Error improper response code. Code is 404
31
Error improper response code. Code is 404
32
Error improper response code. Code is 404
33
Error improper response code. Code is 404
34
Error improper response code. Code is 404
35
Error improper response code. Code is 404
36
Error improper response code. Code is 404
37
Error improper response code. Code is 404
38
Error improper response code. Code is 404
39
Error improper response code. Code is 404
40
Error improper response code. Code is 404
41
Error impr

199
Error improper response code. Code is 404
200
Error improper response code. Code is 404
201
Error improper response code. Code is 404
202
Error improper response code. Code is 404
203
Error improper response code. Code is 404
204
Error improper response code. Code is 404
205
Error improper response code. Code is 404
206
Error improper response code. Code is 404
207
Error improper response code. Code is 404
208
Error improper response code. Code is 404
209
Error improper response code. Code is 404
210
Error improper response code. Code is 404
211
Error improper response code. Code is 404
212
Error improper response code. Code is 404
213
Error improper response code. Code is 404
214
Error improper response code. Code is 404
215
Error improper response code. Code is 404
216
Error improper response code. Code is 404
217
Error improper response code. Code is 404
218
Error improper response code. Code is 404
219
Error improper response code. Code is 404
220
Error improper response code. 

Error improper response code. Code is 404
379
Error improper response code. Code is 404
380
Error improper response code. Code is 404
381
Error improper response code. Code is 404
382
Error improper response code. Code is 404
383
Error improper response code. Code is 404
384
Error improper response code. Code is 404
385
Error improper response code. Code is 404
386
Error improper response code. Code is 404
387
Error improper response code. Code is 404
388
Error improper response code. Code is 404
389
Error improper response code. Code is 404
390
Error improper response code. Code is 404
391
Error improper response code. Code is 404
392
Error improper response code. Code is 404
393
Error improper response code. Code is 404
394
Error improper response code. Code is 404
395
Error improper response code. Code is 404
396
Error improper response code. Code is 404
397
Error improper response code. Code is 404
398
Error improper response code. Code is 404
399
Error improper response code. Code

557
Error improper response code. Code is 404
558
Error improper response code. Code is 404
559
Error improper response code. Code is 404
560
Error improper response code. Code is 404
561
Error improper response code. Code is 404
562
Error improper response code. Code is 404
563
Error improper response code. Code is 404
564
Error improper response code. Code is 404
565
Error improper response code. Code is 404
566
Error improper response code. Code is 404
567
Error improper response code. Code is 404
568
Error improper response code. Code is 404
569
Error improper response code. Code is 404
570
Error improper response code. Code is 404
571
Error improper response code. Code is 404
572
Error improper response code. Code is 404
573
Error improper response code. Code is 404
574
Error improper response code. Code is 404
575
Error improper response code. Code is 404
576
Error improper response code. Code is 404
577
Error improper response code. Code is 404
578
Error improper response code. 

...

Error improper response code. Code is 404
1443
Error improper response code. Code is 404
1444
Error improper response code. Code is 404
1445
Error improper response code. Code is 404
1446
Error improper response code. Code is 404
1447
Error improper response code. Code is 404
1448
Error improper response code. Code is 404
1449
Error improper response code. Code is 404
1450
Error improper response code. Code is 404
1451
Error improper response code. Code is 404
1452
Error improper response code. Code is 404
1453
Error improper response code. Code is 404
1454
Error improper response code. Code is 404
1455
Error improper response code. Code is 404
1456
Error improper response code. Code is 404
1457
Error improper response code. Code is 404
1458
Error improper response code. Code is 404
1459
Error improper response code. Code is 404
1460
Error improper response code. Code is 404
1461
Error improper response code. Code is 404
1462
Error improper response code. Code is 404
1463
Error imprope

Error improper response code. Code is 404
1618
Error improper response code. Code is 404
1619
Error improper response code. Code is 404
1620
Error improper response code. Code is 404
1621
Error improper response code. Code is 404
1622
Error improper response code. Code is 404
1623
Error improper response code. Code is 404
1624
Error improper response code. Code is 404
1625
Error improper response code. Code is 404
1626
Error improper response code. Code is 404
1627
Error improper response code. Code is 404
1628
Error improper response code. Code is 404
1629
Error improper response code. Code is 404
1630
Error improper response code. Code is 404
1631
Error improper response code. Code is 404
1632
Error improper response code. Code is 404
1633
Error improper response code. Code is 404
1634
Error improper response code. Code is 404
1635
Error improper response code. Code is 404
1636
Error improper response code. Code is 404
1637
Error improper response code. Code is 404
1638
Error imprope

Error improper response code. Code is 404
1793
Error improper response code. Code is 404
1794
Error improper response code. Code is 404
1795
Error improper response code. Code is 404
1796
Error improper response code. Code is 404
1797
Error improper response code. Code is 404
1798
Error improper response code. Code is 404
1799
Error improper response code. Code is 404
1800
Error improper response code. Code is 404
1801
Error improper response code. Code is 404
1802
Error improper response code. Code is 404
1803
Error improper response code. Code is 404
1804
Error improper response code. Code is 404
1805
Error improper response code. Code is 404
1806
Error improper response code. Code is 404
1807
Error improper response code. Code is 404
1808
Error improper response code. Code is 404
1809
Error improper response code. Code is 404
1810
Error improper response code. Code is 404
1811
Error improper response code. Code is 404
1812
Error improper response code. Code is 404
1813
Error imprope

1967
Error improper response code. Code is 404
1968
Error improper response code. Code is 404
1969
Error improper response code. Code is 404
1970
Error improper response code. Code is 404
1971
Error improper response code. Code is 404
1972
Error improper response code. Code is 404
1973
Error improper response code. Code is 404
1974
Error improper response code. Code is 404
1975
Error improper response code. Code is 404
1976
Error improper response code. Code is 404
1977
Error improper response code. Code is 404
1978
Error improper response code. Code is 404
1979
Error improper response code. Code is 404
1980
Error improper response code. Code is 404
1981
Error improper response code. Code is 404
1982
Error improper response code. Code is 404
1983
Error improper response code. Code is 404
1984
Error improper response code. Code is 404
1985
Error improper response code. Code is 404
1986
Error improper response code. Code is 404
1987
Error improper response code. Code is 404
1988
Error im

Error improper response code. Code is 404
2143
Error improper response code. Code is 404
2144
Error improper response code. Code is 404
2145
Error improper response code. Code is 404
2146
Error improper response code. Code is 404
2147
Error improper response code. Code is 404
2148
Error improper response code. Code is 404
2149
Error improper response code. Code is 404
2150
Error improper response code. Code is 404
2151
Error improper response code. Code is 404
2152
Error improper response code. Code is 404
2153
Error improper response code. Code is 404
2154
Error improper response code. Code is 404
2155
Error improper response code. Code is 404
2156
Error improper response code. Code is 404
2157
Error improper response code. Code is 404
2158
Error improper response code. Code is 404
2159
Error improper response code. Code is 404
2160
Error improper response code. Code is 404
2161
Error improper response code. Code is 404
2162
Error improper response code. Code is 404
2163
Error imprope

In [47]:
len(list_for_mongo)

1430

### Create a mongoDB and have data stored from scrape 

In [48]:
# DB creation 
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
curlydb = myclient['Naturaly_Curly_db']
curly_collection = curlydb['curly_profiles']

In [49]:
entry2 = curly_collection.insert_many(list_for_mongo)

In [50]:
query_1 = curly_collection.find({})
test = []
for x in query_1:
    test.append(x['signature'])
    #print(x)
print(len(test))    

416869


In [51]:
test_set = set(test)

In [52]:
# Number of unique entries so far
len(test_set)
#7346

7346